In [94]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [95]:
path = "J:\\mimic-iii-clinical-database-1.4\\mimic-iii-clinical-database-1.4"

In [96]:
# load Patients and Diagnoses_icd
patients = pd.read_csv(path + '\\patients.csv')
diagnoses = pd.read_csv(path + '\\diagnoses_icd.csv')
admissions = pd.read_csv(path + '\\admissions.csv')

In [97]:
print(f"Patients shape is {patients.shape}")
print(f"Diagnoses shape is {diagnoses.shape}")
print()

# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Patients shape is (46520, 8)
Diagnoses shape is (651047, 5)

Checking ICD codes:
	ICD codes column dtype is object
	ICD codes are not entirely numeric. Attempting to clean and convert...
	Converted ICD codes to numeric. 97296 rows could not be converted and contain NaN.


In [98]:
# checking why ICD9_CODE columns is Object
diagnoses['ICD9_CODE'].isna().sum()

97296

In [99]:
diagnoses_cleaned = diagnoses.dropna(subset=['ICD9_CODE'])
# Verify the result
print(f"Diagnoses shape after dropping missing ICD9_CODE: {diagnoses_cleaned.shape}")

Diagnoses shape after dropping missing ICD9_CODE: (553751, 5)


In [100]:
# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses_cleaned['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Checking ICD codes:
	ICD codes column dtype is float64
	All ICD codes are numeric.


In [101]:
# now lets extract sepsis patients ( sepsis , severe sepsis , septic shock)
sepsis_codes = {
    99591.0: 'Sepsis',
    99592.0: 'Severe Sepsis',
    78552.0: 'Septic Shock'
}

In [102]:
# Filter for sepsis-related diagnoses
sepsis_diagnoses = diagnoses_cleaned[diagnoses_cleaned['ICD9_CODE'].isin(sepsis_codes.keys())]


In [103]:
sepsis_diagnoses.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [104]:
sepsis_diagnoses.shape

(7770, 5)

In [105]:
# Merge with the patients table to get demographic details
sepsis_patients = sepsis_diagnoses.merge(patients, on='SUBJECT_ID', how='inner')

In [106]:
print(sepsis_patients.head())
sepsis_patients.shape

   ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0      1547         117   164853     16.0    99592.0       108      F   
1      1604         124   138376      6.0    99592.0       114      M   
2       505          64   172056      3.0    99591.0        57      F   
3       679          85   112077     18.0    99591.0        77      M   
4       131          21   111970      2.0    78552.0        18      M   

                   DOB                  DOD             DOD_HOSP  \
0  2083-12-28 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00   
1  2090-11-19 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   
2  2116-06-27 00:00:00                  NaN                  NaN   
3  2090-09-18 00:00:00  2167-09-12 00:00:00  2167-09-12 00:00:00   
4  2047-04-04 00:00:00  2135-02-08 00:00:00  2135-02-08 00:00:00   

               DOD_SSN  EXPIRE_FLAG  
0  2133-12-01 00:00:00            1  
1  2166-02-01 00:00:00            1  
2                  NaN            0  


(7770, 12)

In [107]:

# Sort and deduplicate admissions based on 'subject_id' and 'admittime'
admissions = admissions.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])
admissions.shape


# Merge sepsis patients with admissions to include admission details
# sepsis_patients = sepsis_patients.merge(admissions, on='SUBJECT_ID', how='inner')
# print(sepsis_patients.head())
# sepsis_patients.shape


(58976, 19)

In [108]:
admissions = admissions.drop_duplicates(subset='SUBJECT_ID', keep='first')
admissions.shape

(46520, 19)

In [110]:
# Merge sepsis patients with admissions to include admission details
sepsis_patients = sepsis_patients.merge(admissions, on='SUBJECT_ID', how='inner')
print(sepsis_patients.head())
sepsis_patients.shape


   ROW_ID_x  SUBJECT_ID  HADM_ID_x  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0      1547         117     164853     16.0    99592.0       108      F   
1      1604         124     138376      6.0    99592.0       114      M   
2       505          64     172056      3.0    99591.0        57      F   
3       679          85     112077     18.0    99591.0        77      M   
4       131          21     111970      2.0    78552.0        18      M   

                   DOB                  DOD             DOD_HOSP  ...  \
0  2083-12-28 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00  ...   
1  2090-11-19 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00  ...   
2  2116-06-27 00:00:00                  NaN                  NaN  ...   
3  2090-09-18 00:00:00  2167-09-12 00:00:00  2167-09-12 00:00:00  ...   
4  2047-04-04 00:00:00  2135-02-08 00:00:00  2135-02-08 00:00:00  ...   

  INSURANCE  LANGUAGE  RELIGION  MARITAL_STATUS               ETHNICITY  \
0  Medicaid       NaN  CATHOLIC    

(7770, 30)

In [64]:
# Ensure ADMITTIME is in datetime format
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])

# Identify the first admission for each patient
first_admissions = admissions.sort_values(by=['SUBJECT_ID', 'ADMITTIME']).groupby('SUBJECT_ID').first().reset_index()

# Merge with the sepsis patients to keep only the first admission
sepsis_first_admissions = sepsis_patients.merge(first_admissions[['SUBJECT_ID', 'HADM_ID']], on=['SUBJECT_ID', 'HADM_ID'])

# Display the resulting DataFrame
print(sepsis_first_admissions.head())

# # Save to CSV if needed
# sepsis_first_admissions.to_csv('sepsis_first_admissions.csv', index=False)
# 
# # Check how many patients remain
# unique_patients = sepsis_first_admissions['SUBJECT_ID'].nunique()
# print(f"Total unique sepsis patients with first admission: {unique_patients}")


   ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0       505          64   172056      3.0    99591.0        57      F   
1       307          38   185910      3.0    99592.0        34      M   
2       490          62   116009      4.0    99591.0        55      M   
3      3052         250   124271      2.0    78552.0       235      F   
4      3060         250   124271     10.0    99592.0       235      F   

                   DOB                  DOD             DOD_HOSP DOD_SSN  \
0  2116-06-27 00:00:00                  NaN                  NaN     NaN   
1  2090-08-31 00:00:00                  NaN                  NaN     NaN   
2  2044-05-08 00:00:00                  NaN                  NaN     NaN   
3  2164-12-27 00:00:00  2188-11-22 00:00:00  2188-11-22 00:00:00     NaN   
4  2164-12-27 00:00:00  2188-11-22 00:00:00  2188-11-22 00:00:00     NaN   

   EXPIRE_FLAG  
0            0  
1            0  
2            0  
3            1  
4            1  


In [65]:
sepsis_patients.shape


(7770, 12)

In [21]:
# Ensure DOB and ADMITTIME are in datetime format
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')  # Coerce invalid dates to NaT
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')

# Filter out invalid DOB (e.g., extreme values or NaT)
patients = patients[patients['DOB'].notna() & (patients['DOB'] >= '1900-01-01')]

# Filter out invalid ADMITTIME (e.g., NaT or nonsensical values)
admissions = admissions[admissions['ADMITTIME'].notna() & (admissions['ADMITTIME'] >= '1900-01-01')]

# Merge admissions with patients to include DOB
admissions_with_dob = admissions.merge(patients[['SUBJECT_ID', 'DOB']], on='SUBJECT_ID', how='inner')

# Ensure ADMITTIME is after DOB
admissions_with_dob = admissions_with_dob[admissions_with_dob['ADMITTIME'] >= admissions_with_dob['DOB']]

# Add an upper limit to prevent extreme age calculations (e.g., older than 120 years)
admissions_with_dob = admissions_with_dob[(admissions_with_dob['ADMITTIME'] - admissions_with_dob['DOB']).dt.days <= 120 * 365.25]

# Calculate age at admission (in years)
admissions_with_dob['AGE_AT_ADMISSION'] = (admissions_with_dob['ADMITTIME'] - admissions_with_dob['DOB']).dt.days / 365.25

# Filter for patients aged 18 or older
adults_admissions = admissions_with_dob[admissions_with_dob['AGE_AT_ADMISSION'] >= 18]

# Merge with sepsis data
sepsis_adults = sepsis_patients.merge(adults_admissions[['SUBJECT_ID', 'HADM_ID']], on=['SUBJECT_ID', 'HADM_ID'], how='inner')

# Display and save results
print(sepsis_adults.head())
sepsis_adults.to_csv('sepsis_adult_patients.csv', index=False)

# Check the count of unique adult patients
print(f"Total unique adult sepsis patients: {sepsis_adults['SUBJECT_ID'].nunique()}")


OverflowError: Overflow in int64 addition